# Set up and global variables

In [ ]:
import os
import joblib
import json
import networkx as nx

from pathlib import Path
from itertools import combinations

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from tqdm import tqdm
from scipy.stats import kendalltau

from src.prioritization import *
from src.feature_engineering import add_heuristic_scores, add_feature_sets
from src.ordering import rank_submission, make_baseline_tiebreak, make_baseline_predict

SAMPLE_SIZE = 30

RETRAIN_MODELS = True

In [ ]:
os.environ["CONFIG_ENV"] = "debug"
if False:
    os.environ["CONFIG_ENV"] = "production"

from config import load_config
config = load_config()

DEBUG = config["DEBUG"]
RESOLUTION = config['DEFAULTS']['resolution']
RANDOM_SEED = config['DEFAULTS']['random_seed']

# input data
STORAGE_PATH = config['PATHS']['storage']
HOLD_OUT_DATA_PATH = config['PATHS']['student_hold_out_set']
TRAINED_SCORING_MODELS_PATH = config['PATHS']['hold_out_trained_heuristics']
benchmark_path = config['PATHS']['benchmark_dataset']
FINAL_MODEL_PATH = benchmark_path / "final_teacher_model.pkl"
FINAL_FEATURES_PATH = benchmark_path / "final_selected_features.pkl"
FINAL_HEURISTIC_HYPERPARAMETERS_PATH = benchmark_path / "final_heuristic_selected_features.pkl"

# output data
STUDY_OUTPUT_PATH = HOLD_OUT_DATA_PATH / 'student_study_submissions'
IMAGE_DIR = config['PATHS']['images'] / 'student_study_preparation'

os.makedirs(STUDY_OUTPUT_PATH, exist_ok=True)
os.makedirs(IMAGE_DIR, exist_ok=True)

***

# Loading data

In [ ]:
items = pd.read_csv(STORAGE_PATH / 'items.csv', index_col=0)
defects = pd.read_csv(STORAGE_PATH / f'defects.csv', index_col=0)

log = pd.read_csv(HOLD_OUT_DATA_PATH / 'log.csv', index_col=0, parse_dates=['time'])
defect_log = pd.read_csv(HOLD_OUT_DATA_PATH / 'defect_log.csv', index_col=0)
defect_log.columns = defect_log.columns.astype(int)

In [ ]:
scoring_models = [
    TaskCommonModel,
    TaskCharacteristicModel,
    StudentCommonModel,
    StudentCharacteristicModel,
    StudentEncounteredBeforeModel,
    DefectMultiplicityModel,
    SeverityModel,
]

scoring_models = [model.load(TRAINED_SCORING_MODELS_PATH / f'{model.get_model_name()}.pkl') for model in scoring_models]

scoring_models = {model.get_model_name(): model for model in scoring_models}

In [ ]:
ordering_model = joblib.load(FINAL_MODEL_PATH)
model_columns = joblib.load(FINAL_FEATURES_PATH)

In [ ]:
primary_heuristic, secondary_heuristic = json.load(open(FINAL_HEURISTIC_HYPERPARAMETERS_PATH, "r")).values()

***

# Quality filtering

In [ ]:
# at least two defects
is_non_trivial = (defect_log > 0).sum(axis=1) > 1

***

# Generate all heuristic scores

In [ ]:
if RETRAIN_MODELS:
    # Calculate sampling statistics and remember heuristic scores
    discrete_scores = []
    continuous_scores = []

    # Ensure log is sorted by time
    log = log.sort_values(by='time')

    for idx, submission in tqdm(log.iloc[:500].iterrows() if DEBUG else log.iterrows(), total=log.shape[0], desc="Calculating statistics for sampling"):
        defect_counts = defect_log.loc[idx]

        if not is_non_trivial.loc[idx]:
            for model in scoring_models.values():
                model.update(submission, defect_counts)
            continue

        # Heuristic scores
        discrete_model_scores = {name: model.discretize(submission, defect_counts).dropna() for name, model in scoring_models.items()}
        continuous_model_scores = {name: model._calculate_scores(submission, defect_counts) for name, model in scoring_models.items()}

        # Save model scores
        for defect in defect_counts[defect_counts > 0].index:
            discrete_row = {"submission id": idx, "defect id": defect}
            continuous_row = discrete_row.copy()
            for name, scores in discrete_model_scores.items():
                discrete_row[name] = scores[defect]
                continuous_row[name] = continuous_model_scores[name][defect]
            discrete_scores.append(discrete_row)
            continuous_scores.append(continuous_row)

        for model in scoring_models.values():
            model.update(submission, defect_counts)

    discrete_scores = pd.DataFrame(discrete_scores)
    continuous_scores = pd.DataFrame(continuous_scores)

    discrete_scores.to_csv(HOLD_OUT_DATA_PATH / 'student_holdout_discrete_scores.csv', index=False)
    continuous_scores.to_csv(HOLD_OUT_DATA_PATH / 'student_holdout_continuous_scores.csv', index=False)
else:
    discrete_scores = pd.read_csv(HOLD_OUT_DATA_PATH / 'student_holdout_discrete_scores.csv')
    continuous_scores = pd.read_csv(HOLD_OUT_DATA_PATH / 'student_holdout_continuous_scores.csv')

***

# Construct dataset

In [ ]:
# build the pairwise dataset
long_defects = defect_log.melt(var_name='defect id', value_name='count', ignore_index=False).reset_index(names=['submission id'])
long_defects = long_defects[long_defects['count'] > 0]

def generate_defect_pairs(group):
    """Generate all possible pairs of defects in a submission."""
    defects = group['defect id'].tolist()
    return pd.DataFrame(combinations(defects, 2), columns=['left', 'right'])

all_pairs = (
    long_defects.groupby('submission id')
    .apply(generate_defect_pairs, include_groups=False)
    .reset_index(level=1, drop=True)
    .reset_index()
)

# add metadata
all_pairs['item'] = log.loc[all_pairs['submission id'], 'item'].values

In [ ]:
# add scores
df = add_heuristic_scores(all_pairs, discrete_scores, continuous_scores)
df.dropna(inplace=True)

In [ ]:
# add features

X, _ = add_feature_sets(df, items, defects)

X = X[model_columns]

***

# Calculate predictions

In [ ]:
df['model_probability'] = ordering_model.predict_proba(X)[:, 1]
df['model_prediction'] = ordering_model.predict(X)

df['baseline_tiebreak'] = df.apply(make_baseline_tiebreak(secondary_heuristic), axis=1)
df['baseline_prediction'] = df.apply(make_baseline_predict(primary_heuristic, secondary_heuristic), axis=1)

In [ ]:
agree_on_first = {}
model_rankings = {}
baseline_rankings = {}

for submission_id, submission_df in df.groupby('submission id'):
    model_ranked = rank_submission(submission_df, 'model_prediction', 'model_probability')
    baseline_ranked = rank_submission(submission_df, 'baseline_prediction', 'baseline_tiebreak')
    agree_on_first[submission_id] = model_ranked[0] == baseline_ranked[0]
    model_rankings[submission_id] = model_ranked
    baseline_rankings[submission_id] = baseline_ranked

agree_on_first = pd.Series(agree_on_first)

In [ ]:
eligible = agree_on_first[agree_on_first].index

***

# Sample for the study

In [ ]:
rng = np.random.default_rng(RANDOM_SEED)

# Stratify by topic
strata_labels = items['topic'].unique()
strata_counts = np.zeros(strata_labels.shape[0], dtype=int)
strata = log.loc[eligible, 'item'].map(items['topic'])

# Greedy sampling
sampled_ids = []
while len(sampled_ids) < SAMPLE_SIZE:
    # Least represented strata
    topic_idx = rng.choice(np.where(strata_counts == strata_counts.min())[0], size=1)
    # Collect eligible submissions
    in_strata = strata[strata == strata_labels[topic_idx][0]].index
    unassigned_ids_in_strata = in_strata.difference(sampled_ids)
    try:
        sampled_ids.append(rng.choice(unassigned_ids_in_strata, replace=False))
    except ValueError:
        pass
    strata_counts[topic_idx] += 1

***

# Analysis

## Coverage

In [ ]:
defect_counts = defect_log.loc[sampled_ids].sum().sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(8, 4))
ax.bar(defect_counts.index.astype(str), defect_counts.values)
ax.set_xlabel("Defect ID")
ax.set_ylabel("Number of Submissions")
ax.set_title("Coverage of Defects in Hold-Out Partition")
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig(IMAGE_DIR / "defect_coverage.png", dpi=RESOLUTION)
plt.show()

In [ ]:
task_counts = log.loc[sampled_ids, "item"].value_counts().reindex(items.index).sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(8, 4))
ax.bar(task_counts.index.astype(str), task_counts.values)
ax.set_xlabel("Task ID")
ax.set_ylabel("Number of Submissions")
ax.set_title("Coverage of Tasks in Hold-Out Partition")
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig(IMAGE_DIR / "task_coverage.png", dpi=RESOLUTION)
plt.show()


In [ ]:
topics = items['topic'].unique()
topic_counts = items.loc[log.loc[sampled_ids, "item"], "topic"].value_counts().reindex(topics).sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(topic_counts.index.astype(str), topic_counts.values)
ax.set_xlabel("Topic")
ax.set_ylabel("Number of Submissions")
ax.set_title("Coverage of Topics in Hold-Out Partition")
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig(IMAGE_DIR / "topic_coverage.png", dpi=RESOLUTION)
plt.show()


***

# Generate explanations

***

# Generate survey files

***

# Export